# Generative Agents

This notebook serves as an introduction to the reposetory, explaining the basic structure of the generative agents and showcasing their performance in 
simple examples

Generative Agents are AI agents that use an underlying LLM to react to a current observation that is provided to the in text form.
Due to a memory module observations and actions by the agent are stored and can be used for later decision making, allowing the agent to generate complex emerging behaviors

## Getting started
- Install the required libraries using the gen_agents.yml file
- Create a text file **api_token.txt** that contains the openAI api key

In [9]:
import os
# skip the reading of the file if you already have an environment variable under OPENAI_API_KEY
with open('api_token.txt', 'r') as file:
    api_key = file.read()
os.environ['OPENAI_API_KEY'] = api_key

## Creating an Agent

A generative agents consist of 3 modules: the agent, the longterm memory and a shortterm memory.

Lets create an agent step by step. Afterwards we will use a more convenient method that requires less code.

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_openai import OpenAIEmbeddings
from memory import GenerativeAgentMemory
from gen_agent import GenerativeAgent
from short_term_memory import ShortTermMemory
import faiss
import math

### LTM Module
First we create the LTM module. 
At the core of the module is an LLM embedding vectorstore from the faiss library
This allows the recall of related memories to a given observation.
Around the vectorstore wraps a langchain **TimeWeightedVectorStoreRetriever**
Relevant memories are retrieved according to 3 factors:
- Embedding distance
- Recency (More recent -> higher chance of recall)
- Overall importance of memory (assigned at memory creation)


In [11]:
def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""

    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    # Never questioned this embedding size but it is pretty random
    embedding_size = 1536
    # faiss index from the faiss library
    index = faiss.IndexFlatL2(embedding_size)


    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    #  to a similarity function (0 to 1)
    # ie. with embeddings normalized to [-1,1] max euclidean distance is sqrt(2)
    def relevance_score_fn(score: float) -> float:
        """Return a similarity score on a scale [0, 1]."""
        return 1.0 - score / math.sqrt(2)

    # FAISS is langchain wrapper for a faiss index
    vectorstore = FAISS(
        embeddings_model,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    # k controls the maximum number of documents retrieved per call
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )


# specify the LLM from the openai API used to rank memory importance
# set verbose=True to see what is going on under the hood
ltm_config = {
    'llm_string': 'gpt-4-0125-preview',
    'verbose': True,
}


memory_retriever = create_new_memory_retriever()
long_term_memory = GenerativeAgentMemory(memory_retriever=memory_retriever, **ltm_config)
 

Passing the memory_retriever is not obligatory. The ltm module will create one with the default settings shown here. <br>
For customization create a retriever and pass it. <br>
Passing the retriever parameters to the memory module is not supported but on the TODO list

### STM Module
The short-term memory or conversation memory is much simple.
It keeps track of all the observations and actions by the agent in the current situation (usually a conversation)
and provides a summary of what has happened recently.
It also uses an underlying LMM, which constantly updates the summary of the conversation.

In [12]:
stm_config = {
    'llm_string': 'gpt-4-0125-preview',
    'verbose':True
}
     
short_term_memory = ShortTermMemory(**stm_config)

### Agent Module
The agent integrates information from STM and LTM to react to a given observation. <br>
In addition it has fixed character traits which also influence their behavior
First we need to define a few traits of our agent

**Some notes**:

Current Goal:
This is set and not updated by the agent in the current implementation, as it is not really required for my use case. <br>
For further projects it can be updated by a reflection prompt, where the most recent memories are reviewed

Status: This is a relic from the sims like environment, not sure if its still relevant in this use case. Might be removed

In [13]:
agent_dict = {
    "name": "Brian",
    "age": 52,
    "character_traits": "talkative, extraverted", 
    "communication_style": "midwestern accent, prefers smalltalk",
    "current_goal": "enjoying his weekend",
    "status": "Works at his parents gardening company",
    "initial_observations": [
            "Brian lives in the small town he grew up in",
            "Brian lives in an apartment next door to his parents",
            "Brian works in the family business which provides gardening services",
            "Brian has a close relationship with his parents",
            "Brian gets purpose from and honest day of work",
            "Brian is interested in football and casually plays in the local club",
            "Brian is not up to date with the world news as he feels they don't have influence on his life",
            "Brian plans to take over the family business at some point in his life. He has no ambitions to move away",
             ],
}
 
agent_config = {
    'llm_string': "gpt-4-0125-preview",
    'verbose': True,
}
agent_config.update(agent_dict)



Lets create the agent

In [14]:
brian = GenerativeAgent(short_term_memory=short_term_memory, long_term_memory=long_term_memory, **agent_config)




> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.
Memory: Brian lives in the small town he grew up in
Rating: 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.
Memory: Brian lives in an apartment next door to his parents
Rating: 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate

Because verbose is set to True we can see the LLM calls the LTM makes upon receiving the initial observations. <br>
Each observation or memory gets an importance ranking from 1 to 10

### Convenient Agent Creation

We can also create an agent from a config file in function call.  
You can find some premade agent configs in the folder characters.

In [15]:
import json
path = "characters/brian.txt"
with open(path, 'r') as f:
    agent_dict = json.load(f)

for key, value in agent_dict.items():
    print(f"{key}: {value}")

name: Brian
age: 25
character_traits: talkative, down to earth, enjoys the simple things in live, unpolitical, not the sharpest tool in the shed
communication_style: midwestern accent, prefers jokes and smalltalk
appearance: casual, functionality over appearance
status: Works at his parents company
initial_observations: ['Brian lives in the small town he grew up in', 'Brian lives in an apartment next door to his parents', 'Brian works in the family business which provides gardening services', 'Brian has a close relationship with his parents', 'Brian gets purpose from and honest day of work', 'Brian is interested in football and casually plays in the local club', "Brian is not up to date with the world news as he feels they don't have influence on his life", 'Brian plans to take over the family business at some point in his life. He has no ambitions to move away', 'Brian knows Julia from their local high school. They were close friends', 'Julia moved away to a bigger city. Brian and Jul

These character files contain personal information about the agent.  
There is a convenience function that creates characters from such a file

In [16]:
from utils import create_agent_from_config
agent = create_agent_from_config(path)

You can also pass any additional properties of the agent, stm and ltm to the function.  
It makes it easier to keep the personal information in the character files, while passing agent hyperparameters via the function.  
However properties passed to the function will take precedence if the same property is specified in the character file.



In [17]:
ltm_config = {
    'llm_string': 'gpt-4-0125-preview',
    'verbose': True,
}

stm_config = {
    'llm_string': 'gpt-4-0125-preview',
    'verbose' : True
}

agent_config = {
    'llm_string': "gpt-4-0125-preview",
    'verbose': True,
}
agent_2 = create_agent_from_config(path, agent_config, ltm_config, stm_config)




> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.
Memory: Brian lives in the small town he grew up in
Rating: 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.
Memory: Brian lives in an apartment next door to his parents
Rating: 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate

## Interacting with Agents

Observations or memories can also be added manually

In [18]:
# the return value is not relevant
brian.long_term_memory.add_memory("Brian met Nion last weekend at a football game where they drank a beer together")



> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.
Memory: Brian met Nion last weekend at a football game where they drank a beer together
Rating: 

> Finished chain.


['7ca1b48e-ef32-4402-8c4e-078d6ce85fad']

Now we can for example ask Brain a question.
Generate dialogue response generates a reaction and indicates whether the agent stays in the conversation or says Goodbye

In [19]:
observation = 'Tom says How What do you think about Nion?'
stay_in_dialogue, observation = brian.generate_dialogue_response(observation)
print(observation)

INFO:memory:Fetching relevant memories for given queries
INFO:memory:queries: ["Brian's core characteristics"]


In _generate_reaction: Get current agent summary
In get_summary: Compute new agent summary
In _compute_agent_summary: 


> Entering new LLMChain chain...
Prompt after formatting:
How would you summarize Brian's core characteristics given the following statements:
- [May 29, 2024, 03:52 PM] Brian has a close relationship with his parents
- [May 29, 2024, 03:52 PM] Brian gets purpose from and honest day of work
- [May 29, 2024, 03:52 PM] Brian lives in the small town he grew up in
- [May 29, 2024, 03:52 PM] Brian works in the family business which provides gardening services
- [May 29, 2024, 03:52 PM] Brian is interested in football and casually plays in the local club
- [May 29, 2024, 03:52 PM] Brian plans to take over the family business at some point in his life. He has no ambitions to move away
- [May 29, 2024, 03:52 PM] Brian lives in an apartment next door to his parents
- [May 29, 2024, 03:52 PM] Brian is not up to date with the world news as he feels they don't have influence on 

INFO:memory:Fetching relevant memories for given queries
INFO:memory:queries: ['What is the relationship between Brian and Tom', "Tom is asking for someone's opinion about Nion."]



> Finished chain.
Answer: Tom is asking for someone's opinion about Nion.


> Entering new LLMChain chain...
Prompt after formatting:

            Given the presented context from memory:
            Provide a short summary of the relationship between Brian and Tom
            Provide a short summary of the relevant memories regarding the observation
            Context from memory:
            - [May 29, 2024, 03:52 PM] Brian has a close relationship with his parents
- [May 29, 2024, 03:52 PM] Brian gets purpose from and honest day of work
- [May 29, 2024, 03:53 PM] Brian met Nion last weekend at a football game where they drank a beer together
- [May 29, 2024, 03:52 PM] Brian lives in an apartment next door to his parents
- [May 29, 2024, 03:52 PM] Brian works in the family business which provides gardening services
- [May 29, 2024, 03:52 PM] Brian plans to take over the family business at some point in his life. He has no ambitions to move away
- [May 29, 2024, 03:52 PM] Brian live

INFO:short_term_memory:current summary:Brian thinks Nion is a nice guy, mentioning they both enjoy football after catching up over a beer at a game last weekend. He's curious about Tom's opinion on Nion.



> Finished chain.

> Finished chain.
Brian: "Oh, Nion? Nice guy, we caught up over a beer at the game last weekend. Seems like he's really into football, just like me. How about you, Tom? What's your take on him?"


As verbose is set to True, we get a whole wall of text that explains what is going on under the hood. <br>

- The Info messages are logged by the long-term memory and indicate for which queries relevant memories are fetched.
    - Relevant meaning the smallest euclidean distance to the queries in embedding space 
    - The placement of the info messages are not exactt, they happen when a chain is called that uses queries to LTM
- The green texts are the prompts for every call to the LLM
- The white print statements are infos that indicate in which function the process currently is and what is being done
 
Let's go through it step by step: <br>
- First _generate_reaction is called by generate_dialogue response

**Compute Agent Summary**

- Then _compute agent summary is called
    - This only happens when there is no summary at the creation of a character 
    - or if a certain amount of time has passed, therefore requiring a new summary
    - This is controlled by agent.summary_refresh_seconds, on default it is refreshed once every hour
- Next we see an LMM Prompt that summarizes brian's core characteristics, based on the queried memories
    - The query is displayed in the INFO message from the LTM Module

**Context from memory**
- _summarize_related_memories is called by _generate_reaction
    - this function provides a summary of relevant memories regarding the given observation
- First _get_entity_from_observation is called
    - A prompt identifies the acting entity in the observation, which should be Tom
- Next _get_entity_action is called to describe what the entity is doing in the observation
- This information is used to form two queries for the LTM (info message)
    1. Regarding the relationship between the agent and the entity
    2. Regarding the action of the entity       
- The fetched memories are formatted in the next prompt, which summarizes the information from memory, regarding the relationship and the action of the entity
    - Ideally this should result in no information regarding the relationship of Brain and Tom as no information is present in Memory
    -  Regarding the action of asking about Nion, the summary should mention that Brain met him at a football game last weekend

**Putting the information together**

The next prompt unites all the relevant information about the character and the observation <br>
- First there is general information about the character including the fixed personality traits and communication style <br>
- Then comes the computed agent summary with general information about the agent <br>
- The agents status and the current time (Might move that to the top)
- The summarized memories about the relationship and the content of the observation
- The summary of the conversation so far: empty in this case
- Instructions for generating the characters reaction

Now the reaction is generated <br>
Upon finishing that chain the observation and the agents reaction are stored in the STM module, followed by a prompt that updates the summary 
using the newly added lines.


As that is quite a lot of text, set verbose to False now

In [20]:
brian.verbose = False
brian.long_term_memory.verbose = False
brian.short_term_memory.verbose = False

### STM to LTM Transfer
In our example with brian our question and his reaction were store in his STM memory. <br>
However, it was not transferred to the STM module.

The transfer from STM to LTM is controlled by the beginning and end of a "situation". <br>
In my use case this is usually one conversation with another character, although the boundaries of a situation can be arbitrarily set. <br>
At the start of a situation the STM is cleared (as a precaution for some unrelated residue like our earlier question) <br>
At the end of a situation the information in the STM is summarized and appraised by the character (and also cleared). <br>
Summary and Appraisal are then stored in the LTM module as new memories. <br>
This allows for the separation of more high level information for LTM and more detailed information that might only be relevant in the current situation. <br>
It also allows the agent to reflect on the content of a situation. <br>

Currently, situations are started and ended manually by calling the functions  `agent.start_situation()` and `agent.end_situation()`


## Saving and Loading Agents

If you want your agents to live longer than one python runtime you can save them

In [22]:
from gen_agent import GenerativeAgent
# specify a path for the character to be stored
path = 'saved_characters/brian_test'
GenerativeAgent.save_instance(brian, path)

The saved agent is a folder with a config for the agent, the ltm, the stm and a file with the ltm vectorstore.
Not every part of the agent can be serialized in the same manner. Therefore we store the configurations and the vectorstore and create a new agent object upon loading

In [23]:
! ls saved_characters/brian_test

agent_dict.json  faiss_vector_store.index  ltm_dict.json  stm_dict.json


If you provide an already existing path it will be overwritten. Be careful when dealing with different versions of the same character.


Now we can load the agent.

In [27]:
brian_loaded = GenerativeAgent.load_instance(path)

[Document(page_content='Brian lives in the small town he grew up in', metadata={'importance': 0.045, 'last_accessed_at': datetime.datetime(2024, 5, 29, 15, 53, 39, 870172), 'created_at': datetime.datetime(2024, 5, 29, 15, 52, 48, 169165), 'buffer_idx': 0}), Document(page_content='Brian lives in an apartment next door to his parents', metadata={'importance': 0.045, 'last_accessed_at': datetime.datetime(2024, 5, 29, 15, 53, 39, 870172), 'created_at': datetime.datetime(2024, 5, 29, 15, 52, 49, 326200), 'buffer_idx': 1}), Document(page_content='Brian works in the family business which provides gardening services', metadata={'importance': 0.045, 'last_accessed_at': datetime.datetime(2024, 5, 29, 15, 53, 39, 870172), 'created_at': datetime.datetime(2024, 5, 29, 15, 52, 50, 784044), 'buffer_idx': 2}), Document(page_content='Brian has a close relationship with his parents', metadata={'importance': 0.075, 'last_accessed_at': datetime.datetime(2024, 5, 29, 15, 53, 39, 870172), 'created_at': date

**Note**  
At the moment the loading function creates a new memory retriever with the loaded Faiss index with default configurations, as langchain does not provide save or load functionality for the retrievers.
Therefore retrievers with custom configs need to be recreated manually with the vectorstore from the automatically created default retriever.